In [1]:
from datasets import getNoduleInfoList
nodules = getNoduleInfoList(['sclc'])

/data/kaplinsp/envs/lminfer/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [4]:
import pickle 
path = "/data/kaplinsp/models/log_DenseNet121_2025-04-28_13.35.53.log-fixedsize-tailored-epochs-{fold_index}"
data = {}
for i in range(5):
    for split in ["train", "val"]:
        full_path = f"{path.format(fold_index=i)}/{split}.pkl"
        with open(full_path, "rb") as f:
            data[full_path] = pickle.load(f)
    

In [17]:
set(data['/data/kaplinsp/models/log_DenseNet121_2025-04-28_13.35.53.log-fixedsize-tailored-epochs-0/val.pkl']).intersection(set(data['/data/kaplinsp/models/log_DenseNet121_2025-04-28_13.35.53.log-fixedsize-tailored-epochs-1/train.pkl']))


set()

In [2]:
import multiprocessing as mp
import SimpleITK as sitk

def process_nodule(nodule):
    obj = nodule.image_type(nodule.file_path, nodule.center_lps)
    segmentation_image = obj.nodule_segmentation_image()
    labeled_segmentation_image = sitk.ConnectedComponent(segmentation_image)
    segmentation_id = obj.get_connected_component_id_for_nodule(labeled_segmentation_image)
    return segmentation_id == 0

def multiproc_nodules(nodules):
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.map(
            process_nodule,
            nodules
        )
    return list(zip(nodules, results))

result = multiproc_nodules(nodules)

In [11]:
import pandas as pd 
df = pd.DataFrame(result, columns=["nod", "missing_seg"])
df['file_path'] = df['nod'].apply(lambda x: x.file_path)
df['center'] = df['nod'].apply(lambda x: x.center_lps)

In [12]:
df[df['missing_seg']]

,nod,missing_seg,file_path,center
0,"NoduleInfoTuple(is_nodule=0, nod_id='/data/kap...",True,/data/kaplinsp/transformation/A118.nrrd,"(57.938431912178, 74.21525274590168, -220.1250..."
1,"NoduleInfoTuple(is_nodule=0, nod_id='/data/kap...",True,/data/kaplinsp/transformation/A119.nrrd,"(-62.50294569672131, 48.41508709016394, -81.87..."
3,"NoduleInfoTuple(is_nodule=0, nod_id='/data/kap...",True,/data/kaplinsp/transformation/A124.nrrd,"(85.23157567330207, 40.2135428864168, -104.374..."
7,"NoduleInfoTuple(is_nodule=0, nod_id='/data/kap...",True,/data/kaplinsp/transformation/A134.nrrd,"(-62.71259281264636, 67.56109809133487, -126.875)"
9,"NoduleInfoTuple(is_nodule=0, nod_id='/data/kap...",True,/data/kaplinsp/transformation/A138.nrrd,"(-51.68865945316159, 63.63444642037473, -149.9..."
...,...,...,...,...
143,"NoduleInfoTuple(is_nodule=1, nod_id='/data/kap...",True,/data/kaplinsp/transformation/S462813.nrrd,"(82.62702837417214, -7.168953358278202, -167.5..."
154,"NoduleInfoTuple(is_nodule=1, nod_id='/data/kap...",True,/data/kaplinsp/transformation/SC_12.nrrd,"(61.00500348407647, 32.62281831210191, -221.25)"
158,"NoduleInfoTuple(is_nodule=1, nod_id='/data/kap...",True,/data/kaplinsp/transformation/SC_16.nrrd,"(52.20580560254777, 6.828722494267566, -105.0)"
161,"NoduleInfoTuple(is_nodule=1, nod_id='/data/kap...",True,/data/kaplinsp/transformation/SC_20.nrrd,"(-39.36561137197452, -44.07834411082797, -165.0)"
